In [ ]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace

# %pdb on

# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# NPBのプロセス数
npb_process :list[int] = [2, 4, 8, 16, 32, 64, 128, 256]
train_npb_process :list[int] = npb_process[:-1]
test_npb_process :list[int] = npb_process[-1:]
# NPBのCGの初期変数
cg_na: list[int] = [14000, 30000, 75000, 100000, 1500000]
cg_nonzer: list[int] = [11, 12, 13, 14, 15, 18, 21]
cg_niter: list[int] = [15, 30, 75, 90, 100]
cg_shift: list[int] = [20, 40, 60, 80, 110, 200]

train_cg_na: list[int] = cg_na[:-1]
train_cg_nonzer: list[int] = cg_nonzer[:-1]
train_cg_niter: list[int] = cg_niter[:-1]
train_cg_shift: list[int] = cg_shift[:-1]

test_cg_na: list[int] = cg_na[-1:]
test_cg_nonzer: list[int] = cg_nonzer[-1:]
test_cg_niter: list[int] = cg_niter[-1:]
test_cg_shift: list[int] = cg_shift[-1:]
# NPBのMGの初期変数
mg_size :list[int] = [32, 64, 128, 256, 512]
mg_nit: list[int] = [4, 10, 20, 35, 50]

train_mg_size :list[int] = mg_size[:-1]
train_mg_nit :list[int] = mg_nit[:-1]

test_mg_size :list[int] = mg_size[-1:]
test_mg_nit :list[int] = mg_nit[-1:]


# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48]

test_lulesh_processes: list[int] = [512, 729, 1000]
test_lulesh_iterations: list[int] = [256, 512, 1024]
test_lulesh_sizes: list[int] = [64, 96, 128]

# Extra-Pのオプション
modelerNames: list[str] = [
    # "refining", 
    "multi-parameter",
    "default", 
    # "basic --options poly_exponents=-1,0,1,2,3 log_exponents=0,1 force_combination_exponents=1 allow_negative_exponents=1"
    ]

modelerOption: str = """ --options \#spm=Basic \#spo=poly_exponents=-1,0,1,2,3,log_exponents=0,1,force_combination_exponents=1,allow_negative_exponents=True"""

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]
list_csvDir = [
    "./csv_files/lulesh_1st/",
    "./csv_files/lulesh_2nd/",
    "./csv_files/lulesh_3rd/",
]

In [ ]:
# 学習用のデータ
trainDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=train_npb_process,
    list_na=train_cg_na,
    list_nonzer=train_cg_nonzer,
    list_niter=train_cg_niter,
    list_shift=train_cg_shift,
    csvDir=csvDirPath,
)

# 予測対象用のデータ
testDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=test_npb_process,
    list_na=test_cg_na,
    list_nonzer=test_cg_nonzer,
    list_niter=test_cg_niter,
    list_shift=test_cg_shift,
    csvDir=csvDirPath,
)

# コール回数モデル構築
# 1コール当たりのExclusiveモデル構築

trainDF_cg.head()

In [ ]:
trainDF_mg: pd.DataFrame = return_rawDF_mg(
    list_process=train_npb_process,
    list_problem_size=train_mg_size,
    list_nit=train_mg_nit,
    csvDir=csvDirPath,
)

testDF_mg: pd.DataFrame = return_rawDF_mg(
    list_process=test_npb_process,
    list_nit=test_mg_nit,
    list_problem_size=test_mg_size,
    csvDir=csvDirPath,
)

trainDF_mg.head()

In [ ]:
"""
functionNames_cg list[str]:
    ベンチマークプログラムCGで実行された関数の関数名を保持した文字列のリスト

functionNames_mg list[str]:
    ベンチマークプログラムMGで実行された関数の関数名を保持した文字列のリスト

expVar_cg list[str]:
    ベンチマークプログラムCGの説明変数

expVar_mg list[str]:
    ベンチマークプログラムMGの説明変数

resVar_call str:
    関数コール回数の目的変数

resVar_excl str:
    Exclusiveの目的変数(all)

resVar_excl_perCall str:
    Exclusiveの目的変数(perCall)

dict_symbols_cg dict[str, any]:
    ベンチマークプログラムCGのシンボルを保持した辞書

dict_symbols_mg dict[str, any]:
    ベンチマークプログラムMGのシンボルを保持した辞書

benchmarkName_cg str:
    ベンチマーク名CG

benchmarkName_mg str:
    ベンチマーク名MG

modelerName str:
    Extra-Pのモデル名を保持した文字列

modelerOption str:
    Extra-Pのモデルオプションを保持した文字列

"""
functionNames_cg: list[str] = sorted(list(set(trainDF_cg["Name"].to_list())))
functionNames_mg: list[str] = sorted(list(set(trainDF_mg["Name"].to_list())))
expVar_cg: list[str] = ["process", "na", "nonzer", "niter", "shift"]
expVar_mg: list[str] = ["process", "problem_size", "nit"]
resVar_call: str = "#Call"
resVar_excl: str = "Exclusive"
dict_symbols_cg: dict[str, any] = {}
dict_symbols_mg: dict[str, any] = {}
benchmarkName_cg: str = "cg"
benchmarkName_mg: str = "mg"
modelerName: str = "multi-parameter"
modelerOption: str = """ --options \#spm=Basic \#spo=poly_exponents=-1,0,1,2,3,log_exponents=0,1,force_combination_exponents=1,allow_negative_exponents=True"""

for elem in expVar_cg:
    dict_symbols_cg[elem] = symbols(elem, real=True)
for elem in expVar_mg:
    dict_symbols_mg[elem] = symbols(elem, real=True)

In [ ]:
# CGにおけるモデル構築

"""
dict_cg_model_exclusive_all dict[str, sny]:
    Exclusive予測用のモデル@all to all
dict_cg_model_exclusive_perCall dict[str, any]:
    Exclusive予測用のモデル@perCall
dict_cg_model_call dict[str, any]:
    関数コール予測用のモデル
"""
dict_cg_model_exclusive_all: dict[str, any] = {}
dict_cg_model_exclusive_perCall: dict[str, any] = {}
dict_cg_model_call: dict[str, any] = {}

for functionName in functionNames_cg:

    trainDF_cg_perFunc: pd.DataFrame = trainDF_cg[trainDF_cg["Name"] == functionName]

    cg_model_exclusive_all: str = get_ExtraP_model(
        inputDF_perFunc=trainDF_cg_perFunc,
        expVar=expVar_cg,
        resVar=resVar_excl,
        functionName=functionName,
        dict_symbols=dict_symbols_cg,
        benchmarkName=benchmarkName_cg,
        modelerName=modelerName,
        modelerOption=modelerOption,
    )

    # cg_model_exclusive_perCall :str = get_ExtraP_model(
    #     inputDF_perFunc = trainDF_cg_perFunc,
    #     expVar = expVar_cg,
    #     resVar = resVar_perCall,
    #     functionName = functionName,
    #     dict_symbols = dict_symbols_cg,
    #     benchmarkName = benchmarkName_cg,
    #     modelerName = modelerName,
    #     modelerOption = modelerOption
    # )

    cg_model_call: str = get_ExtraP_model(
        inputDF_perFunc=trainDF_cg_perFunc,
        expVar=expVar_cg,
        resVar=resVar_call,
        functionName=functionName,
        dict_symbols=dict_symbols_cg,
        benchmarkName=benchmarkName_cg,
        modelerName=modelerName,
        modelerOption=modelerOption,
    )